#Mount del drive e installazione/importazione librerie necessarie

In [ ]:
!pip install torch torchvision
!pip install rasterio
!pip install geopandas
!pip install matplotlib
!pip install albumentations
!pip install tqdm
!pip install seaborn


In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
percorso =  "/content/drive/MyDrive/Progetto_laboratorio"
import sys
sys.path.append(percorso)
import fiona
import os
from torch.utils.data import DataLoader, ConcatDataset, Subset

import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF


Mounted at /content/drive


In [3]:
import importlib
import dataset
importlib.reload(dataset)
import model_unet
importlib.reload(model_unet)
from dataset import Dataset
from model_unet import UNET


In [18]:
import utils
importlib.reload(utils)
from utils import (
    save_checkpoint,
    load_checkpoint,
    calculate_metrics_val,
    calculate_metrics_train,
    plot_and_save_metrics,
    save_confusion_matrix,
    plot_and_save_metrics_test
    #find_best_threshold,
)

In [25]:
#Hyperparameters
LEARNING_RATE = [1e-2,1e-3,1e-4,1e-5]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 200
NUM_EPOCHS = 20
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = percorso + '/Dataset/train'
VAL_DIR = percorso + "/Dataset/validation"
TEST_DIR = percorso + "/Dataset/test"



In [5]:
#permette la riproducibilità degli esperimenti, cioè ogni volta che si esegue l'addestramento avremo gli stessi risultati
import random
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False


#Training modello

In [7]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
  train_loss = 0.0
  model.train()                                                    #imposta il modello in training mode
  #loader = loader_aug if use_augmentation else loader_no_aug
  loop = tqdm(loader)                                       #progress bar

  for batch_idx, (data, targets) in enumerate(loop):                       #iteriamo sui batch forniti dal loader
    if (data.shape[1] != 6 and data.shape[3] == 6):
      data = data.permute(0,3,1,2)
    if (data.shape[1] != 6 and data.shape[2] != 6):
      data = data.permute(0,2,1,3)
    data = data.to(device = DEVICE)                                      #sposta i dati sul device , CPU o GPU
    targets = targets.float().unsqueeze(1).to(device = DEVICE)             #convertiamo targets in float e aggiungiamo una dimensione extra
    # forward
    with torch.cuda.amp.autocast():                                     #abilita il mixed precision training
      predictions = model(data)                                          #calcoliamo predizioni
      loss = loss_fn(predictions, targets)                             #calcoliamo la loss

    # backward
    optimizer.zero_grad()                                                  #azzera i gradienti dell'optimizer
    scaler.scale(loss).backward()                                          #scala la loss e calcola i gradienti con backward
    scaler.step(optimizer)                                                 #aggiorna i parametri
    scaler.update()                                                        #aggiorna lo scaler

    train_loss+=loss.item()

    # update tqdm loop
    loop.set_postfix(loss = loss.item())                                     #aggiorna il progress bar con il valore della loss

  train_loss/=len(loader)
  return train_loss



In [8]:

train_transform_aug = A.Compose([
      A.Rotate(limit=35, p=1.0),               #ruota l'immagine di un angolo casuale tra -35 e 35 gradi
      A.HorizontalFlip(p=0.5),                 #esegue un flip orizzontale dell'immagine con probabilità del 50%
      A.VerticalFlip(p=0.1),                   #esegue un flip verticale dell'immagine con probabilità del 10%
      A.Normalize(
          mean=[0.0, 0.0, 0.0],
          std=[1.0, 1.0, 1.0],
          max_pixel_value=255.0,
      ),
      ToTensorV2(),
  ], is_check_shapes=False)

train_transform_no_aug = A.Compose([
      A.Normalize(
          mean=[0.0, 0.0, 0.0],
          std=[1.0, 1.0, 1.0],
          max_pixel_value=255.0,
      ),
      ToTensorV2(),
  ], is_check_shapes=False)

val_transform = A.Compose([
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
], is_check_shapes=False)


In [9]:
#bilanciamento dataset con weightd random sampler, assegna pesi diversi ad ogni campione in base alla frequenza delle classi

train_ds_aug = dataset.Dataset(root = percorso + "/Dataset/train",transform = train_transform_aug,is_train=True)
train_ds_no_aug = dataset.Dataset(root = percorso + "/Dataset/train",transform = train_transform_no_aug,is_train=True)
val_ds = dataset.Dataset(root = percorso + "/Dataset/validation" , transform = val_transform, is_train=False)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=PIN_MEMORY)

print(len(train_ds_aug))
print(len(val_ds))
#Calcola le occorrenze di ogni classe
class_counts = torch.zeros(2)
mask_list = []



15952
384


In [10]:
for i in range(len(train_ds_aug)):
  _, mask = train_ds_aug[i]
  mask_list.append(mask)
  mask_tensor = torch.from_numpy(mask)
  class_counts += torch.bincount(mask_tensor.flatten(), minlength=2)

weights = 1. / class_counts.float()
weights[weights == 0] = 1e-6
weights = weights / weights.sum()
print(weights)
  # Sostituisci i valori negativi con un piccolo valore positivo

# Crea samples_weights
#samples_weights = torch.zeros(len(train_ds), dtype=torch.float)
#for i in range(len(train_ds)):
    #_, mask = train_ds[i]
 #   mask_tensor = torch.from_numpy(mask_list[i])
  #  class_weight = weights[mask_tensor.flatten().unique().to(torch.bool)]
  #  samples_weights[i] = class_weight.mean()
# Crea un sampler


#if torch.any(samples_weights <= 0):
 #   print("Ci sono valori negativi in samples_weights, li correggo...")
 #   samples_weights[samples_weights <= 0] = 1e-6
# Verifica che non ci siano valori negativi o NaN

#sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(train_ds), replacement=True)            #replacement=true permette di scegliere piu volte lo stesso campione e permette l'oversamppling per sovracampionarie le classi minoritarie

# Crea un DataLoader con il sampler
#balanced_train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4, pin_memory=PIN_MEMORY)



tensor([0.1329, 0.8671])


In [11]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0

#Prova del modello

In [28]:

def main():
  output_dir = percorso + "/outputs"
  train_loader_aug = DataLoader(train_ds_aug, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=PIN_MEMORY)
  train_loader_no_aug = DataLoader(train_ds_no_aug, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=PIN_MEMORY)
  combined_dataset = ConcatDataset([train_ds_aug, train_ds_no_aug])
  train_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=PIN_MEMORY)
  subset_indices = np.random.choice(len(train_ds_no_aug), size=int(0.04 * len(train_ds_no_aug)), replace=False)      #replace=False indica che un esempio non puo essere preso piu di una volta
  train_subset = Subset(train_ds_no_aug, subset_indices)

# Creare un DataLoader per il sottoinsieme
  train_subset_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=PIN_MEMORY)

  class_weights = weights[1] / weights[0]                 #per bilanciare le classi con 1 classe minoritaria e 0 classe maggioritaria



  best_f1=0.0
  for lr in LEARNING_RATE:
    print('Proviamo lr : ',lr)
    for weight_decay in [1e-5, 1e-4, 1e-3,1e-2]:
      print('Proviamo weight decay : ',weight_decay)
      for factor in [0.1,0.01]:
        print('Proviamo factor : ',factor)
        model = UNET(in_channels=6, out_channels=1).to(device = DEVICE)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight = class_weights).to(device = DEVICE)
        early_stopping = EarlyStopping(patience=10, min_delta=0.01)
        optimizer = optim.Adam(model.parameters(), lr, weight_decay=weight_decay)         #weight_decay aiuta a prevenire che i pesi diventino troppo grandi aggiungendo una 'penalità' ai pesi
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min",factor=factor,patience=3)                 #riduce il lr di 0.1 quando il f1 score smette di migliorare con pazienza di 3 epoche
        scaler = torch.cuda.amp.GradScaler()                           #utilizzato per mixed precision training, aumenta la velocità e l'efficienza senza perdere di precisione, utilizza mix di float16 e float32
        set_seed(42)

        train_losses=[]
        val_losses=[]
        val_precisions=[]
        val_recalls=[]
        val_f1s=[]
        val_accuracies=[]
        train_precisions=[]
        train_recalls=[]
        train_f1s=[]
        train_accuracies=[]

        scaler = torch.cuda.amp.GradScaler()

        if LOAD_MODEL:
          load_checkpoint(torch.load("model.pth.tar"), model)

        print("Training")

        for epoch in range (NUM_EPOCHS):
          train_loss=train_fn(loader=train_loader,model=model, optimizer=optimizer, loss_fn=loss_fn, scaler=scaler)

          train_losses.append(train_loss)

          val_loss= 0.0
          with torch.no_grad():
            for images,labels in val_loader:
              images = images.to(DEVICE)
              labels = labels.float().unsqueeze(1).to(DEVICE)
              outputs = model(images)
              loss = loss_fn(outputs,labels)
              val_loss+=loss.item()

            val_loss/=len(val_loader)
            val_losses.append(val_loss)

          confusion_matrix_val,accuracy_val,precision_val,recall_val,f1_val = calculate_metrics_val(loader=val_loader,model=model,device=DEVICE)
          val_precisions.append(precision_val)
          val_recalls.append(recall_val)
          val_f1s.append(f1_val)
          val_accuracies.append(accuracy_val)

          accuracy_train,precision_train,recall_train,f1_train = calculate_metrics_train(loader=train_subset_loader,model=model,device=DEVICE)
          train_precisions.append(precision_train)
          train_recalls.append(recall_train)
          train_f1s.append(f1_train)
          train_accuracies.append(accuracy_train)

          print('train loss: ',train_loss)
          print('val_loss : ',val_loss)
          print('train accuracy : ',accuracy_train*100)
          print('train precision : ',precision_train)
          print('train recall : ', recall_train)
          print('train f1 : ', f1_train)
          print('val accuracy : ',accuracy_val*100)
          print('val precision : ',precision_val)
          print('val recall : ', recall_val)
          print('val f1 : ', f1_val)
          save_confusion_matrix(confusion_matrix_val, output_dir, epoch, lr, weight_decay, factor)

          if f1_val > best_f1:
            best_f1 = f1_val
            print(f"Nuovo miglior F1-score: {best_f1}. Salvataggio del modello...")
            model_save_name = 'best_model.pth'
            path = F"/content/drive/MyDrive/Progetto_laboratorio/{model_save_name}"
            torch.save(model.state_dict(), path)

          scheduler.step(val_loss)
          early_stopping(f1_val)

          if early_stopping.early_stop:
            print("Early stopping")
            break

        plot_and_save_metrics(train_losses, val_losses, train_precisions, val_precisions, train_recalls, val_recalls, train_accuracies, val_accuracies, train_f1s, val_f1s, lr, weight_decay, factor, output_dir)
  print("Best F1-score:", best_f1)



In [29]:

if __name__ == "__main__":
  try:
      torch.multiprocessing.set_start_method('spawn', force=True)
  except RuntimeError:
      pass
  torch.cuda.empty_cache()
  main()


Proviamo lr :  0.01
Proviamo weight decay :  1e-05
Proviamo factor :  0.1
Training


100%|██████████| 113/113 [01:56<00:00,  1.03s/it, loss=0.817]


train loss:  0.8718512227049972
val_loss :  0.538002073764801
train accuracy :  98.66397976875305
train precision :  0.5308641975308642
train recall :  0.026704388628208668
train f1 :  0.050850798239274686
val accuracy :  99.21965599060059
val precision :  0.30130505709624794
val recall :  0.03958507469084207
val f1 :  0.06997669969122355
Nuovo miglior F1-score: 0.06997669969122355. Salvataggio del modello...


100%|██████████| 113/113 [01:56<00:00,  1.03s/it, loss=0.739]


train loss:  0.7577643304799511
val_loss :  0.47384769916534425
train accuracy :  98.6407458782196
train precision :  0.4312385397566261
train recall :  0.0446280706596743
train f1 :  0.08088545656354057
val accuracy :  99.18293952941895
val precision :  0.2678082191780822
val recall :  0.05865963694035449
val f1 :  0.09623938536190861
Nuovo miglior F1-score: 0.09623938536190861. Salvataggio del modello...


...

train loss:  0.7015407043220723
val_loss :  0.49111401438713076
train accuracy :  98.66414070129395
train precision :  0.5212355212355212
train recall :  0.03959080872205355
train f1 :  0.07359189366853185
val accuracy :  99.22041893005371
val precision :  0.32363367799113735
val recall :  0.046957714481664845
val f1 :  0.08201538490332964


100%|██████████| 113/113 [01:55<00:00,  1.02s/it, loss=0.665]


train loss:  0.6866938450695139
val_loss :  0.47906801104545593
train accuracy :  98.55936765670776
train precision :  0.36424420421171033
train recall :  0.10055547888490202
train f1 :  0.157602303605467
val accuracy :  99.07745122909546
val precision :  0.21090059430080765
val recall :  0.08898604770783772
val f1 :  0.12516202936122753


  0%|          | 0/113 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
#calcolo la best threshold sui dati di validazione per usarla sui dati di test
from sklearn.metrics import f1_score

def find_best_threshold(model, val_loader):
    best_threshold = 0.5
    best_f1 = 0.0
    for threshold in np.arange(0.1, 1.0, 0.1):
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
                outputs = model(inputs)
                outputs = torch.sigmoid(outputs)
                preds = (outputs > threshold).int()
                all_preds.extend(preds.cpu().numpy().flatten())
                all_labels.extend(labels.cpu().numpy().flatten())
        f1 = f1_score(all_labels, all_preds, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

model = UNET(in_channels=6, out_channels=1).to(device = DEVICE)
best_threshold = find_best_threshold(model, val_loader)
print(best_threshold)

In [ ]:
model = UNET(in_channels=6, out_channels=1).to(device=DEVICE)

# Carica i pesi salvati
model.load_state_dict(torch.load(percorso + '/best_model.pth'))
model.eval()  # Imposta il modello in modalità di valutazione

test_ds = dataset.Dataset(root=percorso + "/Dataset/test", transform=None)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=PIN_MEMORY)

def evaluate_model(loader, model, device):
    model.eval()

    accuracy, precision, recall, f1 = calculate_metrics(test_loader, model)
    return accuracy, precision, recall, f1

# Valuta il modello sul test set
accuracy, precision, recall, f1 = evaluate_model(test_loader, model, DEVICE)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

metrics = accuracy, precision, recall, f1  # Esempio di valori per accuracy, precision, recall, F1-score
plot_and_save_metrics(metrics)

[[5672054  550007]
 [  35068   34327]]
Test Accuracy: 90.70%
Test Precision: 0.0587
Test Recall: 0.4947
Test F1-score: 0.1050
